# 6.S186 Hands-on Tutorial Walk-Through 

In [1]:
!pip install dexhub-api --upgrade
!pip install tensorflow
# Graphics and plotting.
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

In [1]:
# set your API_KEY here
%env DEXHUB_API_TOKEN=32ypM1giGXZ9FfS6dYFrJbLD2z-dvRdfwNhOGW5Xr3U=

env: DEXHUB_API_TOKEN=32ypM1giGXZ9FfS6dYFrJbLD2z-dvRdfwNhOGW5Xr3U=


In [2]:
import dexhub
import os 
import mujoco
from mrl_utils.dataset_conversion import get_sim_and_dataset

/Users/yhpark/anaconda3/envs/mrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1. Load Dataset from DexHub

In [3]:
# Specify the dataset UUID here 

dataset_uuid = "eef4a4db-1359-4230-bc38-a85254525292"

In [4]:
mjmodel, dataset, dim_info = get_sim_and_dataset(dataset_uuid, dataset_name="younghyopark/random", num_next_actions=16)

Dataset eef4a4db-1359-4230-bc38-a85254525292 already downloaded and extracted to ./dataset_eef4a4db-1359-4230-bc38-a85254525292.
Skipping video decoding for simulation data.


Skipping video decoding for simulation data.
Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 784.86ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 858.26ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 561.79ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 995.09ba/s]
Generating train split: 1337 examples [00:00, 279480.91 examples/s]
Compute std:  99%|█████████▉| 167/168 [00:03<00:00, 43.82it/s] 
Returning existing local_dir `/Users/yhpark/.cache/huggingface/lerobot/younghyopark/random810` as remote repo cannot be accessed in `snapshot_download` (None).
Returning existing local_dir `/Users/yhpark/.cache/huggingface/lerobot/younghyopark/random810` as remote repo cannot be accessed in `snapshot_download` (None).


## Step 2. Create a Policy Network 

In [5]:
# TODO: fix that in data preprocessing
dataset.meta.stats['observation.image'] = dataset.meta.stats['observation.images.main']

from lerobot.common.policies.diffusion.configuration_diffusion import DiffusionConfig
from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

input_shapes = {
      "observation.image": dim_info['obs_image'],
      "observation.state": dim_info['obs_state'],
      "observation.environment_state": dim_info['obs_environment_state'],
    }
cfg = DiffusionConfig(n_obs_steps=1, horizon=16, n_action_steps=8, input_shapes=input_shapes, output_shapes={"action": dim_info['action']})
policy = DiffusionPolicy(cfg, dataset_stats=dataset.meta.stats)

## Step 3. Train a Policy 

In [6]:
import torch

training_steps = 5000
device = torch.device("mps")
log_freq = 250

policy.train()
policy.to(device)

optimizer = torch.optim.Adam(policy.parameters(), lr=1e-4)

dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=2,
    batch_size=32,
    shuffle=True,
    pin_memory=device != torch.device("cpu"),
    drop_last=True,
)

In [ ]:
step = 0
done = False
while not done:
    for batch in dataloader:
        batch['observation.image'] = batch['observation.images.main'].unsqueeze(1)

        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        output_dict = policy.forward(batch)
        loss = output_dict["loss"]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % log_freq == 0:
            print(f"step: {step} loss: {loss.item():.3f}")
        step += 1
        if step >= training_steps:
            done = True
            break

step: 0 loss: 1.143


## Step 4. Evaluate your Policy!